In [1]:
require 'torch'
require 'cutorch'
require 'nn'
require 'nnx'
require 'optim'
require 'image'
require 'dataset-mnist'
require 'pl'
require 'paths'

In [2]:
torch.setnumthreads(4)

In [3]:
classes = {'1','2','3','4','5','6','7','8','9','10'}
geometry = {32,32}

### Define model

In [4]:
--torch.setdefaulttensortype("torch.FloatTensor")

In [5]:
require 'cunn'

true	


In [6]:
model = nn.Sequential()
model:add(nn.Reshape(32*32, true))
model:add(nn.Linear(32*32, 10))

In [7]:
parameters,gradParameters = model:getParameters()

In [8]:
criterion = nn.MultiMarginCriterion()

In [9]:
nbTrainingPatches = 60000
nbTestingPatches = 10000

In [10]:
trainData = mnist.loadTrainSet(nbTrainingPatches, geometry)
trainData:normalizeGlobal(mean, std)

<mnist> done	


In [11]:
testData = mnist.loadTestSet(nbTestingPatches, geometry)
testData:normalizeGlobal(mean, std)

<mnist> done	


In [12]:
confusion = optim.ConfusionMatrix(classes)

In [13]:
trainLogger = optim.Logger(paths.concat('train.log'))
testLogger = optim.Logger(paths.concat('test.log'))
finalLogger = optim.Logger(paths.concat('predictions.log'))

In [14]:
batchSize = 100
coefL1 = 0
coefL2 = 0

In [15]:
-- training function
function train(dataset)
   -- epoch tracker
   epoch = epoch or 1

   -- local vars
   local time = sys.clock()

   -- do one epoch
   print('<trainer> on training set:')
   print("<trainer> online epoch # " .. epoch .. ' [batchSize = ' .. batchSize .. ']')
   for t = 1,dataset:size(),batchSize do
      -- create mini batch
      local inputs = torch.Tensor(batchSize,1,geometry[1],geometry[2])
      local targets = torch.Tensor(batchSize)
      local k = 1
      for i = t,math.min(t+batchSize-1,dataset:size()) do
         -- load new sample
         local sample = dataset[i]
         local input = sample[1]:clone()
         local _,target = sample[2]:clone():max(1)
         target = target:squeeze()
         inputs[k] = input
         targets[k] = target
         k = k + 1
      end

      -- create closure to evaluate f(X) and df/dX
      local feval = function(x)
         -- just in case:
         collectgarbage()

         -- get new parameters
         if x ~= parameters then
            parameters:copy(x)
         end

         -- reset gradients
         gradParameters:zero()

         -- evaluate function for complete mini batch
         local outputs = model:forward(inputs)
         local f = criterion:forward(outputs, targets)

         -- estimate df/dW
         local df_do = criterion:backward(outputs, targets)
         model:backward(inputs, df_do)

         -- penalties (L1 and L2):
         if coefL1 ~= 0 or coefL2 ~= 0 then
            -- locals:
            local norm,sign= torch.norm,torch.sign

            -- Loss:
            f = f + coefL1 * norm(parameters,1)
            f = f + coefL2 * norm(parameters,2)^2/2

            -- Gradients:
            gradParameters:add( sign(parameters):mul(coefL1) + parameters:clone():mul(coefL2) )
         end

         -- update confusion
         for i = 1,batchSize do
            confusion:add(outputs[i], targets[i])
         end

         -- return f and df/dX
         return f,gradParameters
      end

      -- optimize on current mini-batch


     -- Perform SGD step:
     sgdState = sgdState or {
        learningRate = .03,
        momentum = 0,
        learningRateDecay = 5e-7
     }
     optim.sgd(feval, parameters, sgdState)

     -- disp progress
     xlua.progress(t, dataset:size())


   end
   
   -- time taken
   time = sys.clock() - time
   time = time / dataset:size()
   print("<trainer> time to learn 1 sample = " .. (time*1000) .. 'ms')

   -- print confusion matrix
   print(confusion)
   trainLogger:add{['% mean class accuracy (train set)'] = confusion.totalValid * 100}
   confusion:zero()

   -- save/log current net
   local filename = paths.concat('mnist.net')
   os.execute('mkdir -p ' .. sys.dirname(filename))
   if paths.filep(filename) then
      os.execute('mv ' .. filename .. ' ' .. filename .. '.old')
   end
   print('<trainer> saving network to '..filename)
   -- torch.save(filename, model)

   -- next epoch
   epoch = epoch + 1
end

In [1]:
finalLogger:setNames{'training error', 'test error'}
-- test function
function test(dataset, flag)
   -- local vars
   local time = sys.clock()

   -- test over given dataset
   print('<trainer> on testing Set:')
   for t = 1,dataset:size(),batchSize do
      -- disp progress
      xlua.progress(t, dataset:size())

      -- create mini batch
      local inputs = torch.Tensor(batchSize,1,geometry[1],geometry[2])
      local targets = torch.Tensor(batchSize)
      local k = 1
      for i = t,math.min(t+batchSize-1,dataset:size()) do
         -- load new sample
         local sample = dataset[i]
         local input = sample[1]:clone()
         local _,target = sample[2]:clone():max(1)
         target = target:squeeze()
         inputs[k] = input
         targets[k] = target
         k = k + 1
      end

      -- test samples
      local preds = model:forward(inputs)

      -- confusion:
      for i = 1,batchSize do
         confusion:add(preds[i], targets[i])
--          if flag==true then finalLogger:add{preds[i],targets[i]} end
      end
   end

   -- timing
   time = sys.clock() - time
   time = time / dataset:size()
   print("<trainer> time to test 1 sample = " .. (time*1000) .. 'ms')

   -- print confusion matrix
   print(confusion)
   testLogger:add{['% mean class accuracy (test set)'] = confusion.totalValid * 100}
    confusion:zero()
end

----------------------------------------------------------------------


In [17]:
i = 0
while (i<10) do
   -- train/test
   train(trainData)
    test(testData)
   i = i + 1
   -- plot errors

      trainLogger:style{['% mean class accuracy (train set)'] = '-'}
      testLogger:style{['% mean class accuracy (test set)'] = '-'}
      trainLogger:plot()
      testLogger:plot()
end

<trainer> on training set:	
<trainer> online epoch # 1 [batchSize = 100]	


Progress: 54401 / 60000	


<trainer> time to learn 1 sample = 0.064350247383118ms	


ConfusionMatrix:
[[    5555       1      50      21      22     139      56      13      55      11]   93.787% 	[class: 1]
 [       2    6409      39      57       5      50      13      18     138      11]   95.061% 	[class: 2]
 [      66      70    5013     119     143      38     131     109     220      49]   84.139% 	[class: 3]
 [      41      50     152    5164      13     340      45      66     181      79]   84.228% 	[class: 4]
 [      14      32      60       9    5215      17      61      27      54     353]   89.267% 	[class: 5]
 [      89      47      65     288     132    4393     112      32     159     104]   81.037% 	[class: 6]
 [      52      29      83       7      92     101    5485       7      57       5]   92.683% 	[class: 7]
 [      49      62     108      42     103      14       6    5553      21     307]   88.635% 	[class: 8]
 [      33     167     105     186      38     253      37      30    4861     141]   83.080% 	[class: 9]
 [      61      51      37   

<trainer> saving network to /home/saurabh/Desktop/COGS 260/Project 2/saurabh/torch tutorial/Assignment 2/mnist.net	
<trainer> on testing Set:	


...

<trainer> time to learn 1 sample = 0.06085033416748ms	
ConfusionMatrix:
[[    5717       1      30      10      12      70      38       7      32       6]   96.522% 	[class: 1]
 [       1    6540      27      28       8      29       4      16      76      13]   97.004% 	[class: 2]
 [      36      55    5329     101      93      25      77      78     145      19]   89.443% 	[class: 3]
 [      22      28     117    5481       4     217      25      55     134      48]   89.398% 	[class: 4]
 [       9      25      39       6    5478       7      49      16      33     180]   93.769% 	[class: 5]
 [      64      33      56     175      62    4739      88      21     130      53]   87.419% 	[class: 6]
 [      35      15      64       1      64      75    5638       3      22       1]   95.269% 	[class: 7]
 [      23      15      79      22      60       7       2    5870      16     171]   93.695% 	[class: 8]
 [      32     108      72     153      24     165      46      18    5160      

unionvalids : FloatTensor - size: 10
  totalValid : 0.92196666666667
}


<trainer> saving network to /home/saurabh/Desktop/COGS 260/Project 2/saurabh/torch tutorial/Assignment 2/mnist.net	
<trainer> on testing Set:	


<trainer> time to test 1 sample = 0.033017492294312ms	


ConfusionMatrix:
[[     956       0       2       1       0      14       5       1       1       0]   97.551% 	[class: 1]
 [       0    1110       2       2       0       1       4       2      14       0]   97.797% 	[class: 2]
 [      10      10     910      15      11      10      13       8      40       5]   88.178% 	[class: 3]
 [       4       1      19     878       0      65       1      15      18       9]   86.931% 	[class: 4]
 [       0       3      10       1     902       3       7       4       7      45]   91.853% 	[class: 5]
 [      10       2       2      18       5     815      13       4      18       5]   91.368% 	[class: 6]
 [      12       3       8       1      10      33     888       2       1       0]   92.693% 	[class: 7]
 [       1       8      21       5       6       3       0     934       1      49]   90.856% 	[class: 8]
 [       8      10       6      21       9      57       9       7     832      15]   85.421% 	[class: 9]
 [      10       8       1   

In [2]:
test(testData, true)

<trainer> on testing Set:	


<trainer> time to test 1 sample = 0.036996293067932ms	
ConfusionMatrix:
[[     956       0       2       1       0      14       5       1       1       0]   97.551% 	[class: 1]
 [       0    1110       2       2       0       1       4       2      14       0]   97.797% 	[class: 2]
 [      10      10     910      15      11      10      13       8      40       5]   88.178% 	[class: 3]
 [       4       1      19     878       0      65       1      15      18       9]   86.931% 	[class: 4]
 [       0       3      10       1     902       3       7       4       7      45]   91.853% 	[class: 5]
 [      10       2       2      18       5     815      13       4      18       5]   91.368% 	[class: 6]
 [      12       3       8       1      10      33     888       2       1       0]   92.693% 	[class: 7]
 [       1       8      21       5       6       3       0     940       1      49]   90.909% 	[class: 8]
 [       8      10       6      21       9      57       9       7     832      

4 : 4
      5 : 5
      6 : 6
      7 : 7
      8 : 8
      9 : 9
      10 : 10
    }
  _prediction : FloatTensor - size: 10
  _pred_idx : LongTensor - size: 1
  nclasses : 10
  _max : FloatTensor - size: 1
  _target : FloatTensor - empty
  unionvalids : FloatTensor - size: 10
  totalValid : 0.91585048970618
}

